# NLP - Sentiment analysis

In [1]:
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import callbacks

In [2]:
def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

In [3]:
word2vec = Word2Vec(sentences=X_train, vector_size=30, window=10)

## Embedding

In [4]:
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

## Baseline model

In [5]:
y_pred = [1]*y_test.shape[0]
y_true = y_test
accuracy_score(y_true, y_pred)

0.508

## RNN

In [6]:
X_train_pad.shape

(2500, 200, 30)

In [8]:
def initilize_model(sha=50):
    model = Sequential()
    model.add(layers.Masking(mask_value=0, input_shape=(200,sha)))
    model.add(layers.SimpleRNN(20))
    model.add(layers.Dense(20, activation='relu'))
    model.add(layers.Dense(1, activation="sigmoid"))
    return model

model = initilize_model(30)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 200, 30)           0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 20)                1020      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 1,461
Trainable params: 1,461
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) # rmsprop

es = callbacks.EarlyStopping(patience=4, restore_best_weights=True)

model.fit(X_train_pad, y_train,
          batch_size=16, # Too small --> no generalization. Too large --> compute slowly
          epochs=20,
          validation_split=0.3,
          callbacks=[es],
          verbose=1)

Epoch 1/20
110/110 [==============================] - 4s 30ms/step - loss: 0.7112 - accuracy: 0.5051 - val_loss: 0.6941 - val_accuracy: 0.5347
Epoch 2/20
110/110 [==============================] - 3s 27ms/step - loss: 0.6842 - accuracy: 0.5594 - val_loss: 0.6877 - val_accuracy: 0.5387
Epoch 3/20
110/110 [==============================] - 3s 27ms/step - loss: 0.6714 - accuracy: 0.5954 - val_loss: 0.6853 - val_accuracy: 0.5640
Epoch 4/20
110/110 [==============================] - 3s 28ms/step - loss: 0.6608 - accuracy: 0.6074 - val_loss: 0.6849 - val_accuracy: 0.5627
Epoch 5/20
110/110 [==============================] - 3s 28ms/step - loss: 0.6499 - accuracy: 0.6297 - val_loss: 0.6812 - val_accuracy: 0.5733
Epoch 6/20
110/110 [==============================] - 3s 27ms/step - loss: 0.6412 - accuracy: 0.6274 - val_loss: 0.6850 - val_accuracy: 0.5813
Epoch 7/20
110/110 [==============================] - 3s 28ms/step - loss: 0.6298 - accuracy: 0.6463 - val_loss: 0.6899 - val_accuracy: 0.5707

In [10]:
model.evaluate(X_test_pad, y_test)

79/79 [==============================] - 1s 7ms/step - loss: 0.7074 - accuracy: 0.5408


[0.7074366211891174, 0.5407999753952026]

## Transfer Learning Word2Vec

In [11]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [12]:
word2vec_transfer = api.load('glove-wiki-gigaword-50')

In [13]:
len(word2vec_transfer.index_to_key)

400000

In [14]:
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

In [15]:
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)

In [16]:
X_train_pad_2.shape

(2500, 200, 50)

In [17]:
model = initilize_model()

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) # rmsprop

es = callbacks.EarlyStopping(patience=4, restore_best_weights=True)

In [18]:
model.fit(X_train_pad_2, y_train,
          batch_size=16,
          epochs=10,
          validation_split=0.3,
          callbacks=[es],
          verbose=1)

Epoch 1/10
110/110 [==============================] - 4s 29ms/step - loss: 0.7402 - accuracy: 0.5063 - val_loss: 0.6985 - val_accuracy: 0.5147
Epoch 2/10
110/110 [==============================] - 3s 28ms/step - loss: 0.6820 - accuracy: 0.5509 - val_loss: 0.6845 - val_accuracy: 0.5467
Epoch 3/10
110/110 [==============================] - 3s 28ms/step - loss: 0.6656 - accuracy: 0.5920 - val_loss: 0.6812 - val_accuracy: 0.5587
Epoch 4/10
110/110 [==============================] - 3s 28ms/step - loss: 0.6489 - accuracy: 0.6183 - val_loss: 0.6731 - val_accuracy: 0.5613
Epoch 5/10
110/110 [==============================] - 3s 28ms/step - loss: 0.6335 - accuracy: 0.6343 - val_loss: 0.6711 - val_accuracy: 0.5760
Epoch 6/10
110/110 [==============================] - 3s 29ms/step - loss: 0.6197 - accuracy: 0.6594 - val_loss: 0.6797 - val_accuracy: 0.5880
Epoch 7/10
110/110 [==============================] - 3s 28ms/step - loss: 0.6060 - accuracy: 0.6680 - val_loss: 0.6661 - val_accuracy: 0.6013

In [19]:
res = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 59.720%
